In [1]:
import pandas as pd
import os
import tqdm
from sklearn.preprocessing import OneHotEncoder
import utils.featurecreations as featurecreations
import importlib
import torch
import torch.nn as nn

In [2]:
# dfs = []
# for file in os.listdir('data'):
#     try:
#         df = pd.read_excel(f'data/{file}')
#         dfs.append(df)
#     except:
#         pass

In [3]:
# data = pd.concat(dfs)

In [4]:
# data = data.reset_index(drop=True)

In [5]:
# data.to_csv('all_races_data.csv')

In [6]:
df = pd.read_feather('all_races_data.fth')

In [7]:
data = df

In [8]:
data['meetingId'] = data['meetingId'].astype(int).astype(str)
data['raceId'] = data['raceId'].astype(int).astype(str)
data['horseId'] = data['horseId'].astype(int).astype(str)

In [9]:
data.columns

Index(['meetingId', 'raceId', 'horseId', 'meetingDate', 'track', 'trackAbbrev',
       'State', 'MPC', 'railPosition', 'TABMeeting', 'isBarrierTrial',
       'hasSectionals', 'resulted', 'raceNo', 'raceName', 'prizeMoney',
       'starters', 'startTime', 'class', 'distance', 'prizeMoneyBreakDown',
       'ageRestrictions', 'SexRestrictions', 'jockeyRestrictions',
       'trackCondition', 'trackConditionNumber', 'raceTime', 'sectional',
       'sectionalDistance', 'PFRaceTime', 'PFRaceL600Time', 'limitWeight',
       'weightType', 'jumps', 'groupStatus', 'tabNo', 'position', 'margin',
       'horse', 'weight', 'barrier', 'inRun', 'flucs', 'priceSP', 'stewards',
       'priceBF', 'gears', 'sex', 'age', 'CareerStarts', 'sire', 'dam',
       'foalDate', 'trainer', 'jockey', 'jockeyId', 'trainerId', 'jock_claim',
       'Reliable', 'RPR', 'RPP', 'WP20R', 'WP20P', 'TP20R', 'TP20P', 'WPTP20R',
       'WPTP20P', 'RSFI', 'ScRank', 'SPrice', 'PFScore', 'PrSe', 'AHSP',
       'RunStyle', 'Publish

In [10]:
race1 = data[data['raceId']=='879736']

In [11]:
race1

,meetingId,raceId,horseId,meetingDate,track,trackAbbrev,State,MPC,railPosition,TABMeeting,...,Pos600,Pos200,Marg600,Marg200,MargFin,Last600Time,Last200Time,ABML600,ABML200,ABMFin
1167,178543,879736,921720,03/09/2021,Albury,ALB,NSW,C,True,True,...,5,3,5.1,1.3,0.0,35.20,12.50,-3.5,0.8,5.4
1168,178543,879736,909170,03/09/2021,Albury,ALB,NSW,C,True,True,...,6,4,5.3,2.5,3.7,35.84,12.96,0.7,3.8,9.8
1169,178543,879736,941305,03/09/2021,Albury,ALB,NSW,C,True,True,...,4,2,3.6,1.1,0.5,35.52,12.60,-1.1,1.7,6.0
1170,178543,879736,884221,03/09/2021,Albury,ALB,NSW,C,True,True,...,3,5,2.5,3.4,6.7,36.86,13.36,7.6,6.3,13.4
1171,178543,879736,917728,03/09/2021,Albury,ALB,NSW,C,True,True,...,2,6,2.4,4.3,11.5,37.68,14.00,13.5,11.0,19.2
1172,178543,879736,893930,03/09/2021,Albury,ALB,NSW,C,True,True,...,1,1,0.0,0.0,3.5,36.68,13.36,6.8,6.6,9.6


In [12]:
# data['isBarrierTrial'].value_counts()

In [13]:
# data['stats_sum'] = data[stats_cols].fillna(0).sum(axis='columns' )

In [14]:
# data['stats_sum']

In [15]:
# data['stats_sum'].value_counts()

In [16]:
stats_cols = ['RPR', 'RPP', 'WP20R', 'WP20P', 'TP20R', 'TP20P', 'WPTP20R',
       'WPTP20P', 'RSFI', 'ScRank', 'SPrice', 'PFScore', 'PrSe', 'AHSP',
       ]

In [17]:
stats = data[stats_cols]

In [18]:
data.RunStyle.head()

0      op/mf
1      op/mf
2    no data
3      mf/bm
4      mf/bm
Name: RunStyle, dtype: object

In [19]:
data.RunStyle.value_counts()

RunStyle
mf         99023
bm         96145
op         67851
op/mf      63275
mf/bm      60258
l          29478
no data    26594
Name: count, dtype: int64

In [20]:
run_style_ohe = OneHotEncoder(sparse_output=False)
run_style_one_hot = run_style_ohe.fit_transform(data[['RunStyle']])
track_ohe = OneHotEncoder(sparse_output=False)
track_one_hot = track_ohe.fit_transform(data[['track']])

In [21]:
data["run_style_one_hot"] = run_style_one_hot.tolist()
data["tracks_one_hot"] = track_one_hot.tolist()

In [22]:
len(data["tracks_one_hot"].iloc[0])

397

In [23]:
horse_form = data.groupby(['horseId'], sort=False, as_index=False)

In [24]:
importlib.reload(featurecreations)

<module 'utils.featurecreations' from 'c:\\Users\\Nick\\Documents\\GitHub\\BetfairSpringCarnival\\src\\utils\\featurecreations.py'>

In [25]:
data = featurecreations.generate_prev_race(data, horse_form)

In [26]:
data['prev_race'].iloc[-1]

'962316'

In [27]:
data['dog_id'] = data['horseId']
data['box'] = data['barrier']
data['bfSP'] = data['priceBF']
data['margin'] = data['MargFin']
data['StartPrice'] = data['priceBF']
data['place'] = data['position']
data['runtime'] = data['ITimeToFin']
data['date'] = pd.to_datetime(data['meetingDate'], format='mixed', dayfirst=True ).dt.date
# data['date'] = pd.Timestamp(data['date'])
data['state'] = data['State']
data['dog_name'] = data['horse']
# data = data[]

In [28]:
# data.barrier?
data.columns
data['State']

0          NZ
1          NZ
2          NZ
3          NZ
4          NZ
         ... 
530176    NSW
530177    NSW
530178    NSW
530179    NSW
530180    NSW
Name: State, Length: 530181, dtype: object

In [29]:
form_only_stats = data[stats_cols].fillna(-1.0)
stats = pd.Series(form_only_stats.values.tolist())

In [30]:
# data = data.drop(columns=stats_cols)
data['stats'] = stats

In [31]:
data = data.query('barrier > 0 and barrier <=20')
data.barrier.value_counts()

barrier
2     59000
1     58993
3     58918
4     58369
5     56513
6     52878
7     47387
8     40527
9     32733
10    25185
11    17305
12    11687
13     5737
14     3425
15      863
16      476
17       94
18       63
19       12
20        9
Name: count, dtype: int64

In [32]:
data['box'] = data['barrier']

In [33]:
data['runtime'] = data['ITimeToFin']
data['race_grade'] = data['class']

In [34]:
data['stats_cols'] = str([stats_cols])
data['stats_cols'] = data['stats_cols'].astype('category')

In [35]:
data = data[~data['isBarrierTrial']]

In [36]:
data = data.reset_index(drop=True)

In [37]:
data.shape

(443872, 108)

In [38]:
race = data[data['raceId']=='879552']

In [39]:
race.barrier

504    7
505    5
506    6
507    2
508    1
509    8
510    4
511    3
Name: barrier, dtype: int64

In [40]:
empty_margin_list = [100]*20
margin_list = [x for x in race["margin"]]
boxes_list = [int(x) for x in race['box']]
for n,x in enumerate(boxes_list):
    empty_margin_list[x-1] = margin_list[n]

In [41]:
print(empty_margin_list)

[0.0, 1.9, 0.2, 1.3, 4.8, 3.9, 1.8, 0.1, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100]


In [42]:
def boosted_softmin(input):
    sft_min = nn.Softmin(dim=-1)
    return sft_min(torch.exp(input))

In [43]:
adjustedMargin = (boosted_softmin(torch.tensor(empty_margin_list)))

In [44]:
adjustedMargin

tensor([3.5963e-01, 1.2204e-03, 2.8820e-01, 2.4922e-02, 0.0000e+00, 3.4271e-22,
        2.3058e-03, 3.2372e-01, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00])

In [45]:
adjustedMargin.to('cuda:0')

tensor([3.5963e-01, 1.2204e-03, 2.8820e-01, 2.4922e-02, 0.0000e+00, 3.4271e-22,
        2.3058e-03, 3.2372e-01, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00], device='cuda:0')

In [46]:
data = data[data['raceId']=='915012']

In [47]:
# data.reset_index(drop=True).to_feather('horsey_racing_v1.fth')

In [56]:
print(data[['place','barrier']])

        place  barrier
235211      7        1
235212      2        8
235213      9       10
235214     11        7
235215      3        2
235216      1        9
235217      5        4
235218      8        3
235219      6        5
235220     10       11
235221      4        6


In [53]:
print(data.shape)
# data = data.

(11, 108)


In [49]:
data.columns

Index(['meetingId', 'raceId', 'horseId', 'meetingDate', 'track', 'trackAbbrev',
       'State', 'MPC', 'railPosition', 'TABMeeting',
       ...
       'bfSP', 'StartPrice', 'place', 'runtime', 'date', 'state', 'dog_name',
       'stats', 'race_grade', 'stats_cols'],
      dtype='object', length=108)

In [50]:
x = data.groupby('track')['barrier'].max()

In [51]:
data.barrier.value_counts()

barrier
1     1
8     1
10    1
7     1
2     1
9     1
4     1
3     1
5     1
11    1
6     1
Name: count, dtype: int64

In [52]:
data.groupby('raceId').count().mean()

meetingId      11.0
horseId        11.0
meetingDate    11.0
track          11.0
trackAbbrev    11.0
               ... 
state          11.0
dog_name       11.0
stats          11.0
race_grade     11.0
stats_cols     11.0
Length: 107, dtype: float64